In [0]:
%skip
df_orders = spark.read.format("csv").option("header",True).option("inferSchema",True).load("/mnt/wwi/bronze/Orders/")

In [0]:
%skip
# Define the input path where new JSON files will arrive
input_path = "/mnt/wwi/bronze/Orders/"

# Define the checkpoint path for Auto Loader to manage state
checkpoint_path = "/mnt/wwi/bronze/orders_autoloader_checkpoints"

# Define the target Delta table name
delta_table_name = "orders"

# Create a sample JSON file for demonstration (optional, for testing)
# dbutils.fs.put(f"{input_path}/data1.json", '{"id": 1, "name": "Alice"}', True)
# dbutils.fs.put(f"{input_path}/data2.json", '{"id": 2, "name": "Bob"}', True)

# Read the stream using Auto Loader
df = (
  spark.readStream
    .format("cloudFiles")
    .option("cloudFiles.format", "csv")
    .option("cloudFiles.schemaLocation", checkpoint_path) # Required for schema inference/evolution
    .load(input_path)
)

# Write the stream to a Delta table
query = (
  df.writeStream
    .format("delta")
    .outputMode("append")
    .option("exactlyOnce", True) # Enable exactly-once processing
    .option("checkpointLocation", checkpoint_path) # Required for exactly-once processing
    .trigger(processingTime="30 seconds") # Process data every 30 seconds
    .start("/mnt/wwi/silver/Orders")
)

# Start the stream (in a real-world scenario, this would likely be part of a job)
# query.awaitTermination() # Uncomment to run indefinitely until manually stopped

**Processing Orders data**

In [0]:
# Define the input path where new JSON files will arrive
input_path = "/mnt/wwi/bronze/Orders/"

# Define the checkpoint path for Auto Loader to manage state
checkpoint_path = "/mnt/wwi/bronze/orders_autoloader_checkpoints"

(spark.readStream
  .format("cloudFiles")
  .option("cloudFiles.format", "csv") # Replace with your file format
  .option("cloudFiles.schemaLocation", checkpoint_path)
  .load(input_path)
  .writeStream
  .format("delta")
  .option("checkpointLocation", checkpoint_path)
  .trigger(availableNow=True) # The key to running as a batch and exiting
  .start("/mnt/wwi/silver/Orders")
)

In [0]:
spark.read.format("delta").load("/mnt/wwi/silver/Orders").count()

%md
**Processing Orders Lines data**

In [0]:
# Define the input path where new JSON files will arrive
input_path = "/mnt/wwi/bronze/OrderLines/"

# Define the checkpoint path for Auto Loader to manage state
checkpoint_path = "/mnt/wwi/bronze/orderlines_autoloader_checkpoints"

(spark.readStream
  .format("cloudFiles")
  .option("cloudFiles.format", "csv") # Replace with your file format
  .option("cloudFiles.schemaLocation", checkpoint_path)
  .load(input_path)
  .writeStream
  .format("delta")
  .option("checkpointLocation", checkpoint_path)
  .trigger(availableNow=True) # The key to running as a batch and exiting
  .start("/mnt/wwi/silver/OrderLines")
)

In [0]:
spark.read.format("delta").load("/mnt/wwi/silver/OrderLines").count()